In [1]:
import sys
import importlib
import os
# make the module available from the src directory
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
from src.dataLoaders.PatientDataLoader import PatientDataProcessor
importlib.reload(sys.modules['src.dataLoaders.PatientDataLoader'])

<module 'src.dataLoaders.PatientDataLoader' from '/Users/askelundsgaard/Documents/datalogi/6-semester/Bachelor/XAI-in-Medical-Image-Analysis/src/dataLoaders/PatientDataLoader.py'>

In [3]:
meta_folder = 'OAIData21/'
data_path = '../datasets/meta_data/' + meta_folder

processor = PatientDataProcessor(base_path=data_path)
meta_data = processor.load_all_clinical_data(labels=["WOMKP"])

In [4]:
from src.dataLoaders.utils.BalancedDataLoader import BalancedBatchSampler
importlib.reload(sys.modules['src.dataLoaders.utils.BalancedDataLoader'])

from src.dataLoaders.NiftiDataLoader2 import NiftiDataLoader
importlib.reload(sys.modules['src.dataLoaders.NiftiDataLoader2'])

from src.transformers.SliceTransformer import SliceAggregateTransform
importlib.reload(sys.modules['src.transformers.SliceTransformer'])
custom_transforms = [SliceAggregateTransform(keys=["image"], slices=9)]

/Users/askelundsgaard/opt/anaconda3/envs/MLA/lib/python3.11/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'dlopen(/Users/askelundsgaard/opt/anaconda3/envs/MLA/lib/python3.11/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN3c1017RegisterOperatorsD1Ev
  Referenced from: <E03EDA44-89AE-3115-9796-62BA9E0E2EDE> /Users/askelundsgaard/opt/anaconda3/envs/MLA/lib/python3.11/site-packages/torchvision/image.so
  Expected in:     <D2077E4D-18BC-34B9-8A9B-1EF634A0F416> /Users/askelundsgaard/opt/anaconda3/envs/MLA/lib/python3.11/site-packages/torch/lib/libtorch_cpu.dylib'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [5]:
dataset = 'niftiShort'
data_path = '../datasets/nifti/'
data_dir = f'{data_path}{dataset}'

# data_dir = 'C:/Users/askel/Downloads/NIFTY/NIFTY/'
# max = 8876
dim = 384
data_loader = NiftiDataLoader(data_dir=data_dir, 
                              meta_data_loader=processor,
                              batch_size=32,
                              spatial_resize=(dim, dim, dim),
                              cache_rate=0.5, 
                              replace_rate=1,
                              custom_transforms=custom_transforms,
                              custom_sampler=False
                              )
# data_loader.load_data(visit_no=visit, subset_size=2058, cache="standard")
# data_loader.load_data(subset_size=84, cache="standard")
#data_loader.load_data(subset_size=4400, cache="standard")
# data_loader.load_data(subset_size=8875, cache="standard")
# data_loader.load_data(cache="persistent")
data_loader.load_data(cache="standard")

File /Users/askelundsgaard/Documents/datalogi/6-semester/Bachelor/XAI-in-Medical-Image-Analysis/src/dataLoaders/saved_data_lists/data_list.pkl does not exist.
Data list loaded: False
Using custom sampler: False
File /Users/askelundsgaard/Documents/datalogi/6-semester/Bachelor/XAI-in-Medical-Image-Analysis/src/dataLoaders/saved_data_lists/data_list.pkl does not exist.
Visits: dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11])
Total images detected: 83
Subset size: 83


Loading dataset: 100%|██████████| 8/8 [00:02<00:00,  3.26it/s]

batch_size: 32


In [11]:
from src.models.medical_models.combined_medical import MedicalCombinedResNetModel
importlib.reload(sys.modules['src.models.medical_models.combined_medical'])

<module 'src.models.medical_models.combined_medical' from '/Users/askelundsgaard/Documents/datalogi/6-semester/Bachelor/XAI-in-Medical-Image-Analysis/src/models/medical_models/combined_medical.py'>

In [12]:
combined_model = MedicalCombinedResNetModel(
        num_epochs=50,
        learning_rate=0.001,
        weight_decay=1e-4,
        data_loader=data_loader,
        dropout_rate=0.1,
        depth=18,
    )

combined_model.train()

Data loader train loader:  3 65
Image spatial dimensions:  torch.Size([32, 9, 384, 384])
Number of input channels:  9
gpu:  mps:0
Is cuda available:  False mps
Number of training images: 65
Epoch 1/50, Train Loss: 11.348268409570059, Val Loss: 14.497766494750977, R^2 Score: -1.5490577220916748
Model saved at /Users/askelundsgaard/Documents/datalogi/6-semester/Bachelor/XAI-in-Medical-Image-Analysis/src/models/medical_models/saved_models/MedicalCombinedResNetModel_18_65_height_384_epoch_1_val_14.5_r2_14.5.pth
Epoch 2/50, Train Loss: 42.04588953653971, Val Loss: 978.3509521484375, R^2 Score: -171.0177459716797
Epoch 3/50, Train Loss: 15.605896612008413, Val Loss: 3769.2841796875, R^2 Score: -661.7312622070312
Epoch 4/50, Train Loss: 3.1187978386878967, Val Loss: 2184.94921875, R^2 Score: -383.1669006347656
Epoch 5/50, Train Loss: 3.0912237552305064, Val Loss: 818.6001586914062, R^2 Score: -142.92970275878906
Epoch 6/50, Train Loss: 2.686594992876053, Val Loss: 300.2662353515625, R^2 Score